# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
# !pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64') # Udacity
env = UnityEnvironment(file_name='Reacher_Windows_x86_64/Reacher') # locally 

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
# env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# while True:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to the environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [7]:
## Watch changes and reload automatically
% load_ext autoreload
% autoreload 2

In [8]:
import pdb
import json
import numpy as np 
import torch 
from collections import deque
from ddpg_agent import Agent
from utils import ensure_dir
import matplotlib.pyplot as plt

import logging
logging.basicConfig(level=logging.INFO, format='')

with open("config.json", "r") as f: 
    config = json.load(f)

In [9]:
agent = Agent(state_size=33, action_size=4, config=config)
brain_name = env.brain_names[0]

def ddpg(agent, 
         brain_name, 
         config, 
         n_episodes=config["trainer"]["num_episodes"]
         ):
    """ Deep Deterministic Policy Gradient """
    
    # Set logger for this function
    logger = logging.getLogger("ddpg")
    
    max_t = 1000
    
    flag = False # When environment is technically solved
    # Save path 
    save_path = config["trainer"]["save_dir"] + config["exp_name"] + "/"
    ensure_dir(save_path)
    scores = [] # list containing scores from each episodes 
    scores_window = deque(maxlen=100)
    
    for i_episode in range(1, n_episodes + 1):
        # reset the environment
        env_info = env.reset(train_mode=True)[brain_name]
        
        # reset noise
        agent.reset()
        
        # get the current state
        state = env_info.vector_observations[0]
        score = 0
        
        for t in range(max_t):
            # choose actions
            action = agent.act(state)
            # send the actions to the environment
            
#             pdb.set_trace()
            
            env_info = env.step(action)[brain_name]
            # get the next state
            next_state = env_info.vector_observations[0]
            # get the reward
            reward = env_info.rewards[0]
            # see if episode has finished
            done = env_info.local_done[0]
            # step 
            agent.step(state, action, reward, next_state, done)
            # accumulate rewards into score variable
            score += reward
            # get next_state and set it to state
            state = next_state
            
            if done: 
                break
            
        # save most recent score
        scores.append(score)
        scores_window.append(score)
        
        print('\rEpisode {}\tAverage Score: {:.3f}'.format(i_episode, np.mean(scores_window)), end="")
        
        if (i_episode % 100 == 0):
            print("\rEpisode {}\tAverage Score: {:.3f}".format(i_episode, \
                                                              np.mean(scores_window)))
        
        # Save occasionnaly 
        if (i_episode % config["trainer"]["save_freq"] == 0):
            torch.save(agent.actor_local.state_dict(), save_path + 
                       "checkpoint_actor_" + str(i_episode) + ".pth")
            torch.save(agent.critic_local.state_dict(), save_path + 
                       "checkpoint_critic_" + str(i_episode) + ".pth")
        
        # Check if envionment solved 
        if not flag:
            if (np.mean(scores_window) >= 30):
                print("\nEnvironment solved in {:d} episodes!\tAverage Score: {:.3f}".format(
                i_episode-100, np.mean(scores_window)))
                # Save solved model 
                torch.save(agent.actor_local.state_dict(), save_path + 
                          "checkpoint_actor_solved.pth")
                torch.save(agent.critic_local.state_dict(), save_path + 
                          "checkpoint_critic_solved.pth")
                flag = True
        
    return scores
    
scores = ddpg(agent=agent, 
              brain_name=brain_name, 
              config=config)

INFO:Agent:Training on gpu
INFO:ReplayBuffer:Training on gpu


Episode 100	Average Score: 1.464
Episode 200	Average Score: 2.940
Episode 300	Average Score: 2.686
Episode 400	Average Score: 1.973
Episode 474	Average Score: 1.498

KeyboardInterrupt: 

In [ ]:
env.close()

In [ ]:
# plot the scores
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.xlabel('Episode number')
plt.ylabel('Score')
plt.show()